# Preprocesamiento Mineria de Datos


*   Estudiante: Inca Cruz Carlos Eduardo

# Setting up PySpark 3.0 on Google Colab

In [1]:
#get package
!wget https://downloads.apache.org/spark/spark-3.2.0/spark-3.2.0-bin-hadoop2.7.tgz
!tar -xvzf spark-3.2.0-bin-hadoop2.7.tgz
!pip install findspark
#env vars
import os
os.environ["SPARK_HOME"] = "/content/spark-3.2.0-bin-hadoop2.7"
#start spark
import findspark
findspark.init()
#import spark session
from pyspark.sql import SparkSession
#build app
spark = SparkSession.builder.appName("PySpark").getOrCreate()
print(spark.sparkContext.appName)
#create sparkcontext
sc=spark.sparkContext
#clear output
from google.colab import output
output.clear()

# 1. Similitud de Cosenos

In [2]:
import numpy as np
from numpy import dot
from numpy.linalg import norm
#Hallar la distancia coseno de dos vectores
def DistanciaCoseno(List1,List2):
    return dot(List1, List2)/(norm(List1)*norm(List2))


In [12]:
# Inicializamos el RDD
numPointsRDD = sc.parallelize([[0.3, 0.8,0.5,0.2],[0.4, 0.4,0.5,0.2],[0.6, 0.1,0.9,0.2]])
# Obtenemos el producto cartesiano
rdd=numPointsRDD.cartesian(numPointsRDD)
# Obtenemos la distancia coseno para cada par de puntos
distRDD = rdd.map(lambda x: (x[0],x[1],str(DistanciaCoseno(x[0],x[1]))))
# Mostramos resultados
print("La distancia coseno entre cada par de puntos es: ")
distRDD.toDF(["Punto 1","Punto 2","Distancia Coseno"]).show()
#Hallar el valor minimo, maximo y la media de dichas distancias
statRDD = distRDD.map(lambda x: float(x[2]))

minv, maxv, meanv = statRDD.min(), statRDD.max(), statRDD.mean()
print ("min:",minv,",max: ", maxv,",mean: ", meanv)

La distancia coseno entre cada par de puntos es: 
+--------------------+--------------------+------------------+
|             Punto 1|             Punto 2|  Distancia Coseno|
+--------------------+--------------------+------------------+
|[0.3, 0.8, 0.5, 0.2]|[0.3, 0.8, 0.5, 0.2]|               1.0|
|[0.3, 0.8, 0.5, 0.2]|[0.4, 0.4, 0.5, 0.2]|0.9254604351544817|
|[0.3, 0.8, 0.5, 0.2]|[0.6, 0.1, 0.9, 0.2]|0.6723280987166992|
|[0.4, 0.4, 0.5, 0.2]|[0.3, 0.8, 0.5, 0.2]|0.9254604351544817|
|[0.6, 0.1, 0.9, 0.2]|[0.3, 0.8, 0.5, 0.2]|0.6723280987166992|
|[0.4, 0.4, 0.5, 0.2]|[0.4, 0.4, 0.5, 0.2]|               1.0|
|[0.4, 0.4, 0.5, 0.2]|[0.6, 0.1, 0.9, 0.2]|0.8925774123174453|
|[0.6, 0.1, 0.9, 0.2]|[0.4, 0.4, 0.5, 0.2]|0.8925774123174453|
|[0.6, 0.1, 0.9, 0.2]|[0.6, 0.1, 0.9, 0.2]|               1.0|
+--------------------+--------------------+------------------+

min: 0.6723280987166992 ,max:  1.0 ,mean:  0.8867479880419169


In [21]:
# Inicializamos el RDD
numPointsRDD = sc.parallelize(np.random.randint(1,20,(15,15)).tolist())
# Obtenemos el producto cartesiano
rdd=numPointsRDD.cartesian(numPointsRDD)
# Obtenemos la distancia coseno para cada par de puntos
distRDD = rdd.map(lambda x: (x[0],x[1],str(DistanciaCoseno(x[0],x[1]))))
print(distRDD.collect())
# Mostramos resultados
print("La distancia coseno entre cada par de puntos es: ")
distRDD.toDF(["Punto 1","Punto 2","Distancia Coseno"]).show()
#Hallar el valor minimo, maximo y la media de dichas distancias
statRDD = distRDD.map(lambda x: float(x[2]))

minv, maxv, meanv = statRDD.min(), statRDD.max(), statRDD.mean()
print ("min:",minv,",max: ", maxv,",mean: ", meanv)

[([18, 12, 5, 4, 3, 9, 5, 5, 17, 5, 7, 13, 9, 6, 1], [18, 12, 5, 4, 3, 9, 5, 5, 17, 5, 7, 13, 9, 6, 1], '1.0000000000000002'), ([18, 12, 5, 4, 3, 9, 5, 5, 17, 5, 7, 13, 9, 6, 1], [1, 8, 17, 2, 2, 13, 11, 17, 19, 4, 11, 3, 19, 5, 5], '0.7303845722311336'), ([18, 12, 5, 4, 3, 9, 5, 5, 17, 5, 7, 13, 9, 6, 1], [17, 1, 12, 3, 5, 12, 12, 13, 5, 3, 2, 6, 19, 18, 10], '0.7350689632694482'), ([18, 12, 5, 4, 3, 9, 5, 5, 17, 5, 7, 13, 9, 6, 1], [17, 12, 6, 17, 6, 12, 10, 15, 2, 6, 5, 16, 6, 13, 16], '0.7693895879980359'), ([18, 12, 5, 4, 3, 9, 5, 5, 17, 5, 7, 13, 9, 6, 1], [6, 6, 19, 11, 1, 6, 17, 5, 19, 18, 3, 11, 18, 18, 3], '0.7603040287103172'), ([18, 12, 5, 4, 3, 9, 5, 5, 17, 5, 7, 13, 9, 6, 1], [9, 1, 14, 7, 2, 8, 10, 8, 9, 12, 15, 7, 14, 19, 3], '0.7518393468290364'), ([18, 12, 5, 4, 3, 9, 5, 5, 17, 5, 7, 13, 9, 6, 1], [10, 15, 10, 14, 16, 11, 10, 6, 3, 11, 17, 9, 15, 18, 3], '0.7591442017721693'), ([1, 8, 17, 2, 2, 13, 11, 17, 19, 4, 11, 3, 19, 5, 5], [18, 12, 5, 4, 3, 9, 5, 5, 17, 5, 7, 

# Binarizacion

In [14]:
import numpy as np
# Crear rdd categorico
catPointsRDD = sc.parallelize(enumerate([['alto', 'caro', 'azul'],
                             ['medio', 'caro', 'verde'],
                             ['alto', 'barato', 'azul'],
                             ['medio', 'caro', 'vermelho'],
                             ['baixo', 'barato', 'verde'],
                            ]))
# obtener claves unicas
clavesRDD = (catPointsRDD
             .flatMap(lambda x: [((x[0],xi),1) for xi in x[1]])
             .reduceByKey(lambda x,y: x)
             .map(lambda x: x[0])
             )

claves = dict((v,k) for k,v in clavesRDD.collect())
nclaves = len(claves)
print (claves, nclaves)
def CreateNP(atributos,claves):  
    #Binarizar utilizando diccionario de claves
    
    array = np.zeros(len(claves))
    for atr in atributos:
        array[ claves[atr] ] = 1
    return array

# Converte o RDD para o formato binário, utilizando o dict chaves
binRDD = catPointsRDD.map(lambda rec: (rec[0],CreateNP(rec[1], claves)))
binRDD.collect()

{'alto': 2, 'caro': 0, 'baixo': 4, 'verde': 1, 'azul': 2, 'medio': 3, 'barato': 4, 'vermelho': 3} 8


[(0, array([1., 0., 1., 0., 0., 0., 0., 0.])),
 (1, array([1., 1., 0., 1., 0., 0., 0., 0.])),
 (2, array([0., 0., 1., 0., 1., 0., 0., 0.])),
 (3, array([1., 0., 0., 1., 0., 0., 0., 0.])),
 (4, array([0., 1., 0., 0., 1., 0., 0., 0.]))]

# 2.Normalizacion

In [16]:
import math
# Modulo para normalizar un vector
def nor(x):
    cociente=math.sqrt(sum([i**2 for i in x]))
    return [i/cociente for i in x]

In [17]:
#EJEMPLO 2: NORMALIZAR ARREGLOS EN UN RDD
numPointsRDD = sc.parallelize([[0.37454012, 0.95071431,7,77,0.73199394, 0.59865848,22,-222,0.73199394, 0.59865848,22,-222],[0.37454012, 0.95071431,7,77,0.73199394, 0.59865848,22,-222,0.73199394, 0.59865848,22,-222]])
distRDD = numPointsRDD.map(lambda x: (x,nor(x)))
print(distRDD.collect())
#Mostrar Dataset
distRDD.toDF(["Punto","Punto Normalizado"]).show()

[([0.37454012, 0.95071431, 7, 77, 0.73199394, 0.59865848, 22, -222, 0.73199394, 0.59865848, 22, -222], [0.0011530216774382237, 0.0029267737952364714, 0.021549498467794494, 0.23704448314573945, 0.002253443184066408, 0.0018429699996417401, 0.06772699518449699, -0.6834269514071969, 0.002253443184066408, 0.0018429699996417401, 0.06772699518449699, -0.6834269514071969]), ([0.37454012, 0.95071431, 7, 77, 0.73199394, 0.59865848, 22, -222, 0.73199394, 0.59865848, 22, -222], [0.0011530216774382237, 0.0029267737952364714, 0.021549498467794494, 0.23704448314573945, 0.002253443184066408, 0.0018429699996417401, 0.06772699518449699, -0.6834269514071969, 0.002253443184066408, 0.0018429699996417401, 0.06772699518449699, -0.6834269514071969])]
+--------------------+--------------------+
|               Punto|   Punto Normalizado|
+--------------------+--------------------+
|[0.37454012, 0.95...|[0.00115302167743...|
|[0.37454012, 0.95...|[0.00115302167743...|
+--------------------+--------------------+

In [20]:
#EJEMPLO 3: NORMALIZAR ARREGLOS EN UN RDD
numPointsRDD = sc.parallelize(np.random.randint(1,20,(15,15)).tolist())
distRDD = numPointsRDD.map(lambda x: (x,nor(x)))
print(distRDD.collect())
#Mostrar Dataset
distRDD.toDF(["Punto","Punto Normalizado"]).show()

[([16, 1, 7, 12, 9, 6, 14, 6, 18, 13, 2, 19, 10, 15, 7], [0.35502997425484983, 0.022189373390928115, 0.15532561373649678, 0.26627248069113735, 0.19970436051835302, 0.13313624034556867, 0.31065122747299356, 0.13313624034556867, 0.39940872103670605, 0.2884618540820655, 0.04437874678185623, 0.42159809442763413, 0.22189373390928113, 0.3328406008639217, 0.15532561373649678]), ([6, 17, 19, 5, 19, 6, 10, 2, 1, 18, 5, 11, 14, 3, 1], [0.1380496377687712, 0.3911406403448517, 0.43715718626777544, 0.11504136480730932, 0.43715718626777544, 0.1380496377687712, 0.23008272961461865, 0.04601654592292373, 0.023008272961461865, 0.4141489133063136, 0.11504136480730932, 0.2530910025760805, 0.32211582146046613, 0.0690248188843856, 0.023008272961461865]), ([7, 6, 13, 14, 4, 17, 19, 19, 8, 6, 1, 12, 4, 3, 14], [0.15880407399252633, 0.13611777770787972, 0.294921851700406, 0.31760814798505266, 0.09074518513858648, 0.3856670368389925, 0.4310396294082858, 0.4310396294082858, 0.18149037027717296, 0.136117777707879

# 3.N-grams

In [9]:
#Codigo en python
def obtenNGramas(sentence, n):
    l1=[x for x in sentence.split(" ")]
    return [[x for x in sentence.split(" ")][i:i+3] for i in range(len([x for x in sentence.split(" ")])-(3-1))]
# Prueba
listaPalabras='this is a foo bar sentences and i want to ngramize it'
obtenNGramas(listaPalabras, 3)

[['this', 'is', 'a'],
 ['is', 'a', 'foo'],
 ['a', 'foo', 'bar'],
 ['foo', 'bar', 'sentences'],
 ['bar', 'sentences', 'and'],
 ['sentences', 'and', 'i'],
 ['and', 'i', 'want'],
 ['i', 'want', 'to'],
 ['want', 'to', 'ngramize'],
 ['to', 'ngramize', 'it']]

In [ ]:
dataRDD=sc.parallelize(['this is a foo bar sentences and i want to ngramize it','this is a foo bar sentences and i want to ngramize it','this is a foo bar sentences and i want to ngramize it'],8)
dataRDD2 = dataRDD.map(lambda x: (x,str(obtenNGramas(x,3))))
print(dataRDD2.collect())


[('this is a foo bar sentences and i want to ngramize it', "[['this', 'is', 'a'], ['is', 'a', 'foo'], ['a', 'foo', 'bar'], ['foo', 'bar', 'sentences'], ['bar', 'sentences', 'and'], ['sentences', 'and', 'i'], ['and', 'i', 'want'], ['i', 'want', 'to'], ['want', 'to', 'ngramize'], ['to', 'ngramize', 'it']]"), ('this is a foo bar sentences and i want to ngramize it', "[['this', 'is', 'a'], ['is', 'a', 'foo'], ['a', 'foo', 'bar'], ['foo', 'bar', 'sentences'], ['bar', 'sentences', 'and'], ['sentences', 'and', 'i'], ['and', 'i', 'want'], ['i', 'want', 'to'], ['want', 'to', 'ngramize'], ['to', 'ngramize', 'it']]"), ('this is a foo bar sentences and i want to ngramize it', "[['this', 'is', 'a'], ['is', 'a', 'foo'], ['a', 'foo', 'bar'], ['foo', 'bar', 'sentences'], ['bar', 'sentences', 'and'], ['sentences', 'and', 'i'], ['and', 'i', 'want'], ['i', 'want', 'to'], ['want', 'to', 'ngramize'], ['to', 'ngramize', 'it']]")]


# 4.Escalonamiento

In [ ]:
# Modulo Escalonamiento
def Escalonamiento(x):
    return [(i-min(x))/(max(x)-min(x)) for i in x]

#scale xi = (xi - minimum x) / (maximum x - minimum x)
#Crear RDD
dataRDD=sc.parallelize([[1,2,3],[4,5,6],[7,8,9],[10,11,12]],2)
# Escalonar
dataRDD2 = dataRDD.map(lambda x: (str(x),str(Escalonamiento(x))))
print(dataRDD2.collect())
#Mostrar Dataset
dataRDD2.toDF(["Punto","Punto Escalonado"]).show()

[0.0, 0.5, 1.0]
[('[1, 2, 3]', '[0.0, 0.5, 1.0]'), ('[4, 5, 6]', '[0.0, 0.5, 1.0]'), ('[7, 8, 9]', '[0.0, 0.5, 1.0]'), ('[10, 11, 12]', '[0.0, 0.5, 1.0]')]
+------------+----------------+
|       Punto|Punto Escalonado|
+------------+----------------+
|   [1, 2, 3]| [0.0, 0.5, 1.0]|
|   [4, 5, 6]| [0.0, 0.5, 1.0]|
|   [7, 8, 9]| [0.0, 0.5, 1.0]|
|[10, 11, 12]| [0.0, 0.5, 1.0]|
+------------+----------------+



In [24]:
def Escalonamiento(x):
    return [(i-min(x))/(max(x)-min(x)) for i in x]

print(Escalonamiento([10,11,12]))
#scale xi = (xi - minimum x) / (maximum x - minimum x)

dataRDD=sc.parallelize(np.random.randint(1,20,(15,3)).tolist(),2)
dataRDD2 = dataRDD.map(lambda x: (str(x),Escalonamiento(x)))
print(dataRDD2.collect())
#Mostrar Dataset
dataRDD2.toDF(["Punto","Punto Escalonado"]).show()

[0.0, 0.5, 1.0]
[('[9, 9, 19]', [0.0, 0.0, 1.0]), ('[12, 10, 5]', [1.0, 0.7142857142857143, 0.0]), ('[9, 9, 19]', [0.0, 0.0, 1.0]), ('[3, 7, 8]', [0.0, 0.8, 1.0]), ('[6, 6, 2]', [1.0, 1.0, 0.0]), ('[18, 10, 1]', [1.0, 0.5294117647058824, 0.0]), ('[2, 5, 18]', [0.0, 0.1875, 1.0]), ('[18, 18, 16]', [1.0, 1.0, 0.0]), ('[10, 19, 3]', [0.4375, 1.0, 0.0]), ('[14, 7, 14]', [1.0, 0.0, 1.0]), ('[4, 6, 6]', [0.0, 1.0, 1.0]), ('[19, 17, 14]', [1.0, 0.6, 0.0]), ('[1, 9, 9]', [0.0, 1.0, 1.0]), ('[16, 7, 4]', [1.0, 0.25, 0.0]), ('[2, 15, 4]', [0.0, 1.0, 0.15384615384615385])]
+------------+--------------------+
|       Punto|    Punto Escalonado|
+------------+--------------------+
|  [9, 9, 19]|     [0.0, 0.0, 1.0]|
| [12, 10, 5]|[1.0, 0.714285714...|
|  [9, 9, 19]|     [0.0, 0.0, 1.0]|
|   [3, 7, 8]|     [0.0, 0.8, 1.0]|
|   [6, 6, 2]|     [1.0, 1.0, 0.0]|
| [18, 10, 1]|[1.0, 0.529411764...|
|  [2, 5, 18]|  [0.0, 0.1875, 1.0]|
|[18, 18, 16]|     [1.0, 1.0, 0.0]|
| [10, 19, 3]|  [0.4375, 1.0, 0.0]|

# 5. Estandarizacion

In [ ]:
import statistics as stats
def Estandarizacion(x):
    return [(i-(sum(x)/len(x)))/(stats.variance(x)) for i in x]

#scale xi = (xi - minimum x) / (maximum x - minimum x)
#Crear RDD
dataRDD=sc.parallelize([[1,2,3],[4,5,6],[7,8,9],[10,11,12]],2)
#Estandarizar vectores
dataRDD2 = dataRDD.map(lambda x: (str(x),str(Estandarizacion(x))))
#Mostrar RDD
print(dataRDD2.collect())
dataRDD2.toDF(["Punto","Punto Estandarizado"]).show()

[-1.0, 0.0, 1.0]
[('[1, 2, 3]', '[-1.0, 0.0, 1.0]'), ('[4, 5, 6]', '[-1.0, 0.0, 1.0]'), ('[7, 8, 9]', '[-1.0, 0.0, 1.0]'), ('[10, 11, 12]', '[-1.0, 0.0, 1.0]')]
+------------+----------------+
|       Punto|Punto Escalonado|
+------------+----------------+
|   [1, 2, 3]|[-1.0, 0.0, 1.0]|
|   [4, 5, 6]|[-1.0, 0.0, 1.0]|
|   [7, 8, 9]|[-1.0, 0.0, 1.0]|
|[10, 11, 12]|[-1.0, 0.0, 1.0]|
+------------+----------------+



In [27]:
import statistics as stats
def Estandarizacion(x):
    return [(i-(sum(x)/len(x)))/(stats.variance(x)) for i in x]


#Crear RDD
dataRDD=sc.parallelize(np.random.randint(1,20,(15,3)).tolist(),2)
#Estandarizar vectores
dataRDD2 = dataRDD.map(lambda x: (str(x),str(Estandarizacion(x))))
#Mostrar RDD
print(dataRDD2.collect())
dataRDD2.toDF(["Punto","Punto Estandarizado"]).show()


[('[4, 4, 3]', '[1.0000000000000004, 1.0000000000000004, -1.9999999999999996]'), ('[11, 16, 2]', '[0.026490066225165573, 0.12582781456953643, -0.15231788079470196]'), ('[18, 17, 9]', '[0.13698630136986303, 0.09589041095890412, -0.23287671232876708]'), ('[12, 3, 8]', '[0.21311475409836061, -0.22950819672131145, 0.016393442622950803]'), ('[3, 19, 11]', '[-0.125, 0.125, 0.0]'), ('[18, 3, 15]', '[0.09523809523809523, -0.14285714285714285, 0.047619047619047616]'), ('[19, 8, 9]', '[0.1891891891891892, -0.10810810810810811, -0.08108108108108109]'), ('[3, 15, 6]', '[-0.1282051282051282, 0.1794871794871795, -0.05128205128205128]'), ('[14, 10, 7]', '[0.29729729729729726, -0.027027027027027077, -0.27027027027027034]'), ('[8, 9, 18]', '[-0.12087912087912085, -0.08791208791208789, 0.2087912087912088]'), ('[4, 4, 10]', '[-0.16666666666666666, -0.16666666666666666, 0.3333333333333333]'), ('[7, 3, 17]', '[-0.038461538461538464, -0.11538461538461539, 0.15384615384615385]'), ('[7, 12, 14]', '[-0.3076923